# Data Retrieval Scripts
I combined data from several sources + fixed some rows manually. Here are links to the sources:

- UCI Name-Gender Dataset: https://archive.ics.uci.edu/ml/datasets/Gender+by+Name
    - Downloaded the dataset and matched the names I had in my own dataset. If match found, prob = 1.
- Chinese-English Names-Genders Corpus: https://raw.githubusercontent.com/wainshine/Chinese-Names-Corpus/master/English_Names_Corpus/English_Cn_Name_Corpus_Gender（48W）.txt

- Genderize.io: https://genderize.io/
    - Querying names one at a time. There is a daily limit to appx 1500 queries per IP-address.

In [66]:
!pip install genderize
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 3.8 MB/s eta 0:00:00a 0:00:01


In [67]:
import pandas as pd
from tqdm import tqdm
from unidecode import unidecode
from genderize import Genderize # Gender API

In [171]:
# Import the df
names_genders = pd.read_csv('data/names_genders.csv', sep = ',')

# Sort to have the NAs for updating ready
## This sort is performed every time when the data is read in
names_genders = names_genders.sort_values(['gender', 'prob', 'first_name'], 
                          ascending = [0,0,1], 
                          na_position= 'first').reset_index(drop = True)
print(f"Number of names uncategorized: {names_genders['gender'].isnull().sum()}")
names_genders.head()

Number of names uncategorized: 9189


,first_name,alph_value,gender,prob
0,JanikVasily,J,NaN,NaN
1,JannFrederik,J,NaN,NaN
2,Jannatun,J,NaN,NaN
3,Jannes,J,NaN,NaN
4,Janniele,J,NaN,NaN


# Match the data with external data

In [172]:
def search_gender_from_data(names_genders, ext_data, ext_firstname, ext_gender):
    """Match the names with an external dataset and retrieve gender
    Args:
        ext_data (pd.DataFrame): external dataset to be used for matching
        ext_firstname (str): name of the 'first_name' column in the external dataset
        ext_gender (str): name of the 'gender' column in the external dataset
    """
    
    # Search for names from the UCI name data set
    for i in tqdm(range(len(names_genders))):
        
        # If gender is already marked, ignore the row
        if names_genders.loc[i, 'gender'] == 'M' or names_genders.loc[i, 'gender'] == 'F':
                pass
        else:
            # Extract the name and letter
            firstname = names_genders.loc[i, 'first_name']

            # Search in a subset of the externalm dataset
            idx = ext_data[ext_data[ext_firstname] == firstname].index

            # If no index found, no name -> do nothing
            if len(idx) == 0:
                pass
            
            else:
                idx = idx.values[0]
              #  print(firstname, ext_data.loc[idx, ext_gender], 1)
                names_genders.loc[i, 'gender'] = ext_data.loc[idx, ext_gender] # get the gender
                names_genders.loc[i, 'prob'] = 1 # set prob to 1

### Use the Chinese data to fetch names

In [ ]:
# Import the data (see below for data retrieval)
chinese_names = pd.read_csv('chinese_names.csv')

# Run the function
search_gender_from_data(names_genders, chinese_names, 'name_eng', 'gender')

# Sort the data so that no-retrieved data is first
names_genders = names_genders.sort_values(['gender', 'prob', 'first_name'], 
                                               ascending = [0,0,1], 
                                               na_position= 'first').reset_index(drop = True)
# Update the names-genders csv
names_genders.to_csv('data/names_genders.csv', index = False)

# Remove the Chinese names tables
! rm chinese_names.csv

### Use the UCI data

In [169]:
# Uncomment the below code
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

# Import the data (see below for data retrieval)
uci_names = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00591/name_gender_dataset.csv')

# Run the function
search_gender_from_data(names_genders, uci_names, 'Name', 'Gender')

# Sort the data so that no-retrieved data is first
names_genders = names_genders.sort_values(['gender', 'prob', 'first_name'], 
                                               ascending = [0,0,1], 
                                               na_position= 'first').reset_index(drop = True)
# Update the names-genders csv
names_genders.to_csv('data/names_genders.csv', index = False)

  0%|                                       | 45/21336 [00:00<01:36, 221.39it/s]

Jaromir M 1
Jaroslaw M 1


  3%|█▏                                    | 678/21336 [00:03<01:33, 221.64it/s]

Jorgen M 1
Jozsef M 1


  4%|█▌                                    | 883/21336 [00:04<01:33, 219.55it/s]

...

 45%|████████████████▋                    | 9636/21336 [00:43<00:53, 219.57it/s]

Havard M 1


 46%|█████████████████                    | 9863/21336 [00:44<00:52, 217.23it/s]

Ildiko F 1


100%|████████████████████████████████████| 21336/21336 [00:45<00:00, 471.81it/s]


# Pull the genders from Genderize.io API

In [35]:
# A helper function to update data from APIs, etc

def update_names_table(names_genders, limit = 500):
    
    try: 
        # For-loop querying the genderize.io API
        for i in tqdm(range(limit)):

            # Extract the name
            first_name = names_genders.loc[i, 'first_name'] # first name

            # Check if the name has already been checked
            ## Query only if the name hasn't been checked already
            if names_genders.loc[i, 'prob'] >= 0 and names_genders.loc[i, 'prob'] <= 1:
                pass

            else:
                # Try querying the Genderize.io API
                gender_info = Genderize().get([first_name])
                names_genders.loc[i, 'gender'] = gender_info[0]['gender']
                names_genders.loc[i,'prob'] = gender_info[0]['probability']

    except:
        print(f'Iteration nr {i}')
        print('Limit likely exceeded.')

     # Sort the data so that no-retrieved data is first
    names_genders = names_genders.sort_values(['gender', 'prob', 'first_name'], 
                                               ascending = [0,0,1], 
                                               na_position= 'first').reset_index(drop = True)
    # Write to csv once data is pulled
    names_genders.to_csv('data/names_genders.csv', index = False)

In [36]:
# Run the function that querys names from Genderize.io
update_names_table(names_genders, 200)

 94%|██████████████████████████████████████▎  | 187/200 [03:51<00:16,  1.24s/it]

Iteration nr 187
Limit likely exceeded.


In [117]:
# A sloppy solution for the Chinese Names data:

# Clone the Github repo
!git clone https://github.com/wainshine/Chinese-Names-Corpus

# Import the data
chinese_names = pd.read_table('./Chinese-Names-Corpus/English_Names_Corpus/English_Cn_Name_Corpus_Gender（48W）.txt',
                             sep = '|',
                            names=['name_chi', 'name_eng', 'gender']
                         )

# Remove rows that do not include alphabetical letters
chinese_names = chinese_names[chinese_names['name_eng'].str.contains('[A-Za-z]', na=False)].reset_index(drop = True)
chinese_names = chinese_names[~(chinese_names['name_eng'].str.len() < 4)].reset_index(drop = True)
chinese_names['name_eng'] = chinese_names['name_eng'].str.replace(' ', '')

# Drop duplicates
chinese_names = chinese_names.drop_duplicates('name_eng').reset_index(drop = True)

# # Select only the English version and gender
chinese_names = chinese_names[['name_eng', 'gender']]

# Save to csv
chinese_names.to_csv('chinese_names.csv', index = False)

# Remove the directory, as it is not needed anymore
!rm -rf Chinese-Names-Corpus

Cloning into 'Chinese-Names-Corpus'...
remote: Enumerating objects: 266, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 266 (delta 26), reused 44 (delta 25), pack-reused 213
Receiving objects: 100% (266/266), 35.93 MiB | 7.47 MiB/s, done.
Resolving deltas: 100% (141/141), done.
